# ANN - PyTorch

In [1]:
import pandas as pd
import numpy as np
import pickle
import networkx as nx
import matplotlib.pyplot as plt

import math
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from random import shuffle

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

### Data processing

In [2]:
# Import database
database = pickle.load( open( "Mod_AnyT_DB.p", "rb" ) )

#### Data Normalization

In [3]:
def norm_diams(ls_diams):
    avail_diams = [6, 8, 10, 12, 14] #inches
    min_diam = min(avail_diams)
    max_diam = max(avail_diams)
    n_d = (ls_diams-min_diam)/(max_diam-min_diam)
    return n_d


In [4]:
x_diameters = []
y_res_index = []
for i in range(len(database)):
    x_diameters.append(norm_diams(database.loc[i]['Diams']))
    y_res_index.append(database.loc[i]['avgPrPa'])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(x_diameters, y_res_index, random_state=1, test_size=0.2 )
print(len(X_train), len(y_test))

4000 1000


In [6]:
X_train, X_test, y_train, y_test = torch.tensor(X_train, dtype = torch.float32), torch.tensor(X_test, dtype = torch.float32), torch.tensor(y_train, dtype = torch.float32), torch.tensor(y_test, dtype = torch.float32)

### ANN model definition

In [7]:
class MLP(nn.Module):
    def __init__(self, in_features, h_sizes, out_size):
        super(MLP, self).__init__()
        self.hidden_linear = nn.ModuleList()
        #self.hidden_dropout = nn.ModuleList()
        self.h_len = len(h_sizes)
        
        #Add initial layer
        self.hidden_linear.append(nn.Linear(in_features, h_sizes[0]))
        #self.hidden_dropout.append(nn.Dropout(p=0.5))
        
        #Add hidden layers        
        for k in range(len(h_sizes)-1):
            self.hidden_linear.append(nn.Linear(h_sizes[k], h_sizes[k+1]))
        #    self.hidden_dropout.append(nn.Dropout(p=0.5))
        
        #Add output layer
        self.hidden_linear.append(nn.Linear(h_sizes[-1], out_size))
        
        
    def forward(self, x):
        y = x
        for i in range(len(self.hidden_linear)-1):
            y = self.hidden_linear[i](y)
        #    y = self.hidden_dropout[i](y)
            y = torch.tanh(y) #F.relu(y)
        
        y = self.hidden_linear[-1](y)
        y = torch.tanh(y)
            
        return y

In [8]:
mlp = MLP(40, [50, 50, 50], 1)

In [9]:
#mlp(torch.rand(1,40))

In [10]:
# # Hyperparameters
batch_size = 50
lr = 0.001
weight_decay = 8e-4
num_epochs = 1500

In [11]:
data_loader_X_train = torch.utils.data.DataLoader(
    X_train,
    shuffle=False,
    batch_size = batch_size
)

data_loader_y_train = torch.utils.data.DataLoader(
    y_train,
    shuffle = False,
    batch_size = batch_size
)

In [12]:
optimizer = torch.optim.Adam(mlp.parameters(), lr=lr, weight_decay=weight_decay, amsgrad = True)

In [13]:
mlp.train()
f_loss = nn.MSELoss() #(preds-y_train[i])**2

for epoch in range(num_epochs):
    total_loss= 0
#     total_correct =0
    
    iter_y_train = iter(data_loader_y_train)
    iter_X_train = iter(data_loader_X_train)
    
    #Data
    for batch in iter_X_train:
        #Prediction
        preds = mlp(batch)

        #Calculate the loss
        loss = f_loss(preds.reshape(batch_size,1), next(iter_y_train).reshape(batch_size,1))
        
        total_loss += loss
    
    #Backpropagate
    optimizer.zero_grad() #To avoid adding up gradients
    total_loss.backward() #calculate gradients

    #Optimizer step
    optimizer.step() #Update weights
        
    if epoch%50 == 0:
        print("epoch ", epoch, " total loss  ", total_loss )
    
    if epoch == num_epochs-1:
        print("epoch ", epoch, " total loss  ", total_loss )

C:\Programs\Anaconda3\envs\PyTorch_Env\lib\site-packages\torch\autograd\__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


epoch  0  total loss   tensor(9.7284, grad_fn=<AddBackward0>)
epoch  50  total loss   tensor(0.7755, grad_fn=<AddBackward0>)
epoch  100  total loss   tensor(0.4969, grad_fn=<AddBackward0>)
epoch  150  total loss   tensor(0.4501, grad_fn=<AddBackward0>)
epoch  200  total loss   tensor(0.4337, grad_fn=<AddBackward0>)
epoch  250  total loss   tensor(0.4220, grad_fn=<AddBackward0>)
epoch  300  total loss   tensor(0.4097, grad_fn=<AddBackward0>)
epoch  350  total loss   tensor(0.3909, grad_fn=<AddBackward0>)
epoch  400  total loss   tensor(0.3566, grad_fn=<AddBackward0>)
epoch  450  total loss   tensor(0.2909, grad_fn=<AddBackward0>)
epoch  500  total loss   tensor(0.1911, grad_fn=<AddBackward0>)
epoch  550  total loss   tensor(0.1397, grad_fn=<AddBackward0>)
epoch  600  total loss   tensor(0.1180, grad_fn=<AddBackward0>)
epoch  650  total loss   tensor(0.1047, grad_fn=<AddBackward0>)
epoch  700  total loss   tensor(0.0954, grad_fn=<AddBackward0>)
epoch  750  total loss   tensor(0.0888, gra

In [14]:
num_params = []
i = 0
        
for parameter in mlp.parameters():
    if i%2 == 0:
        num_params.append(parameter.reshape(-1,1).shape[0])
    else:
        num_params[-1] += parameter.reshape(-1,1).shape[0]
    i += 1

print(num_params)

[2050, 2550, 2550, 51]


In [16]:
mlp(X_test[1:10]).t()

tensor([[ 0.4151,  0.0534,  0.3322,  0.3964,  0.3218,  0.3639,  0.4098,  0.4682,
         -0.1262]], grad_fn=<TBackward>)

In [17]:
y_test[1:10]

tensor([ 0.4086,  0.0567,  0.3450,  0.4014,  0.3403,  0.3413,  0.4215,  0.4311,
        -0.1600])

In [18]:
#R squared 
mlp.eval()
r2_score( y_train.detach().numpy() , mlp(X_train).detach().numpy().flatten() )

0.9543690375054529

In [19]:
#R squared
mlp.eval()
r2_score( y_test.detach().numpy() , mlp(X_test).detach().numpy().flatten() )

0.9597847269113402

In [20]:
x_real_values_train = y_train.detach().numpy()
y_pred_values_train = mlp(X_train).detach().numpy().flatten()

x_real_values_test = y_test.detach().numpy()
y_pred_values_test = mlp(X_test).detach().numpy().flatten()

In [21]:
import plotly.graph_objects as go

In [22]:
fig = go.FigureWidget(data=go.Scatter(x=x_real_values_train, y=y_pred_values_train,  line={'dash': 'dash'}, name = 'Train'))

fig.add_trace(go.Scatter(x=x_real_values_test, y=y_pred_values_test,  line={'dash': 'dash'}, name='Test'))
#fig.add_trace(go.Scatter(x=x_axis, y=first_DB_pd['minPrPa'],  line={'dash': 'dash'}, name='Min. Res.'))


fig.update_traces(mode='markers',  marker_size=3)
fig.update_layout(title='Pred. vs Actual - Prasad & Park Resilience Index',
                    xaxis_title="Real Index",
                    yaxis_title="Pred. Index",
                    legend_title="Legend",
                    autosize=False,
                    width=800,
                    height=500,
                    font=dict(
                        #family="Courier New, monospace",
                        size=18,
    ))
#yaxis_zeroline=False, xaxis_zeroline=False)
fig.update_layout(shapes=[
dict(
  type= 'line',
  yref= 'y', y0= -0.30, y1= 1,   # adding a horizontal line at Y = 1
  xref= 'x', x0= -0.30, x1= 1
     ) 
])

fig.update_xaxes(range=[-0.3, 1])
fig.update_yaxes(range=[-0.3, 1])

fig

FigureWidget({
    'data': [{'line': {'dash': 'dash'},
              'marker': {'size': 3},
              'mod…

In [23]:
import pickle

In [24]:
pickle.dump(mlp, open('ANN_PyTorch_96.p', 'wb'))